In [1]:
import pandas as pd
import numpy as np

def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

def InfoGain(data,feature,tname):
    te = entropy(data[tname])
    vals,counts= np.unique(data[feature],return_counts=True)
    we =np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[feature]==vals[i]).dropna()[tname]) for i in range(len(vals))]) #same line
    Information_Gain = te - we
    return Information_Gain

def ID3(data,f,tname,pnode):
    if len(np.unique(data[tname])) == 1:
        return np.unique(data[tname])[0]
    elif len(f) ==0:
        return pnode
    else:
        pnode = np.unique(data[tname])[np.argmax(np.unique(data[tname])[1])]
        item_values = [InfoGain(data,feature,tname) for feature in f]
        bfi= np.argmax(item_values)
        bf = f[bfi]
        tree = {bf:{}}
        f = [i for i in f if i != bf]
        for value in np.unique(data[bf]):
            sub_data = data.where(data[bf] == value).dropna()
            subtree = ID3(sub_data,f,tname,pnode)
            tree[bf][value] = subtree
        return(tree)

def predict(query,tree,default = 1):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]]
            except:
                return default
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result

dataset = pd.read_csv('p3.csv')
tdata = dataset.sample(frac=0.8)
print("The Training Dataset:\n",tdata)

f=['Outlook','Temperature','Humidity','Wind']
tname="PlayTennis"
pnode=None
tree=ID3(tdata,f,tname,pnode)
print("\nTree:\n",tree)

query=dataset.iloc[:,:].to_dict(orient="records")

#Testing
print("\nQuery:\n",query[10])
result=predict(query[10],tree,1.0)
print("\n\nTesting sample 1:",query[10],"PREDICTED =>",result)
result=predict(query[12],tree,1.0)
print("\nTesting sample 2:",query[12],"PREDICTED =>",result)






The Training Dataset:
      Outlook Temperature Humidity    Wind PlayTennis
12  Overcast         Hot   Normal    Weak        Yes
10     Sunny        Mild   Normal  Strong        Yes
4       Rain        Cool   Normal    Weak        Yes
0      Sunny         Hot     High    Weak         No
13      Rain        Mild     High  Strong         No
3       Rain        Mild     High    Weak        Yes
5       Rain        Cool   Normal  Strong         No
9       Rain        Mild   Normal    Weak        Yes
2   Overcast         Hot     High    Weak        Yes
11  Overcast        Mild     High  Strong        Yes
7      Sunny        Mild     High    Weak         No

Tree:
 {'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

Query:
 {'Outlook': 'Sunny', 'Temperature': 'Mild', 'Humidity': 'Normal', 'Wind': 'Strong', 'PlayTennis': 'Yes'}


Testing sample 1: {'Outlook': 'Sunny', 'Temperature': 'Mild', 'Humidity': 'No